# Baseline Model using TF-IDF for binary

## Set up

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import load_npz
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score


In [3]:
PATH = Path.cwd().parents[2]
DATA_PATH = os.path.join(PATH, 'data/processed/ir_gigahorse')

## List Traditional ML model to compare

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier

models = {
  "LogisticRegression()": LogisticRegression(),
  "DecisionTreeClassifier()": DecisionTreeClassifier(),
  "RandomForestClassifier()": RandomForestClassifier(),
  "AdaBoostClassifier()": AdaBoostClassifier(),
  "ExtraTreesClassifier()": ExtraTreesClassifier(),
  "XGBClassifier()": XGBClassifier(),
  "LGBMClassifier()": LGBMClassifier(),
  "SVC()": SVC(),
  "GaussianNB()": GaussianNB(),
  "KNeighborsClassifier()": KNeighborsClassifier(),
  "SGDClassifier()": SGDClassifier(),
  "MLPClassifier()": MLPClassifier(),
}

## Load Dataset

In [5]:
X = load_npz(os.path.join(DATA_PATH, "tfidf_vector_opcode.npz"))
# X = load_npz(os.path.join(DATA_PATH, "tfidf_vector_grouping_opcode.npz"))
y = np.load(os.path.join(DATA_PATH, "labels.npy"))

# Optional: stratify on dominant class (if multilabel → use IterativeStratification later)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# grouping_X_train, grouping_X_val, grouping_y_train, grouping_y_val = train_test_split(grouping_X, y, test_size=0.2, random_state=42)

In [6]:
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4968 stored elements and shape (67, 143)>

In [7]:
y[:, 0]

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1])

In [8]:
y[:, 1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0])

In [9]:
y[:, 2]

array([0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0])

## Prepare Features and Labels

In [10]:
y_mint = y[:, 0]
y_leak = y[:, 1]
y_limit = y[:, 2]

## Train/Test Split

In [11]:
X_mint_train, X_mint_test, y_mint_train, y_mint_test = train_test_split(
    X, y_mint, test_size=0.2, random_state=42
)

X_leak_train, X_leak_test, y_leak_train, y_leak_test = train_test_split(
    X, y_leak, test_size=0.2, random_state=42
)

X_limit_train, X_limit_test, y_limit_train, y_limit_test = train_test_split(
    X, y_limit, test_size=0.2, random_state=42
)

## Run all models and collect reports

## Show report

In [12]:
data = {
  'mint': {
    "X_train": X_mint_train,
    "X_test": X_mint_test,
    "y_train": y_mint_train,
    "y_test": y_mint_test
  },
  'leak': {
    "X_train": X_leak_train,
    "X_test": X_leak_test,
    "y_train": y_leak_train,
    "y_test": y_leak_test
  },
  'limit': {
    "X_train": X_limit_train,
    "X_test": X_limit_test,
    "y_train": y_limit_train,
    "y_test": y_limit_test
  }
}

reports = []

for d in data.values():
    X_train = d['X_train']
    X_test = d['X_test']
    y_train = d['y_train']
    y_test = d['y_test']

    # Convert to dense for models that require it
    def maybe_dense(model, X):
        # List of models that require dense input
        dense_models = (GaussianNB, MLPClassifier)
        if isinstance(model, dense_models):
            return X.toarray()
        return X

    def get_report_all_ml_dense(X_train, y_train, X_test, y_test):
        report_list = []
        for name, model in models.items():
            X_train_mod = maybe_dense(model, X_train)
            X_test_mod = maybe_dense(model, X_test)
            model.fit(X_train_mod, y_train)
            y_pred = model.predict(X_test_mod)
            report_dict = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
            avg_scores = report_dict["macro avg"]
            report_list.append({
                "Model": name,
                "Precision": avg_scores["precision"],
                "Recall": avg_scores["recall"],
                "F1-score": avg_scores["f1-score"]
            })
        df_report = pd.DataFrame(report_list)
        df_report = df_report.sort_values("F1-score", ascending=False).reset_index(drop=True)
        return df_report

    df_report = get_report_all_ml_dense(X_train, y_train, X_test, y_test)
    reports.append(df_report)

[LightGBM] [Info] Number of positive: 15, number of negative: 38
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000155 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1276
[LightGBM] [Info] Number of data points in the train set: 53, number of used features: 81
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.283019 -> initscore=-0.929536
[LightGBM] [Info] Start training from score -0.929536
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

In [13]:
reports[0]

,Model,Precision,Recall,F1-score
0,GaussianNB(),0.742424,0.700,0.714286
1,RandomForestClassifier(),0.884615,0.625,0.634783
2,ExtraTreesClassifier(),0.884615,0.625,0.634783
3,XGBClassifier(),0.884615,0.625,0.634783
4,DecisionTreeClassifier(),0.588889,0.600,0.590643
5,KNeighborsClassifier(),0.625000,0.575,0.575758
6,SVC(),0.357143,0.500,0.416667
7,LogisticRegression(),0.357143,0.500,0.416667
8,SGDClassifier(),0.357143,0.500,0.416667
9,LGBMClassifier(),0.357143,0.500,0.416667


In [14]:
reports[1]

,Model,Precision,Recall,F1-score
0,LogisticRegression(),0.428571,0.5,0.461538
1,DecisionTreeClassifier(),0.428571,0.5,0.461538
2,RandomForestClassifier(),0.428571,0.5,0.461538
3,AdaBoostClassifier(),0.428571,0.5,0.461538
4,ExtraTreesClassifier(),0.428571,0.5,0.461538
5,XGBClassifier(),0.428571,0.5,0.461538
6,LGBMClassifier(),0.428571,0.5,0.461538
7,SVC(),0.428571,0.5,0.461538
8,GaussianNB(),0.428571,0.5,0.461538
9,KNeighborsClassifier(),0.428571,0.5,0.461538


In [15]:
reports[2]

,Model,Precision,Recall,F1-score
0,ExtraTreesClassifier(),0.770833,0.788889,0.775401
1,DecisionTreeClassifier(),0.714286,0.733333,0.708333
2,XGBClassifier(),0.714286,0.733333,0.708333
3,RandomForestClassifier(),0.714286,0.733333,0.708333
4,LGBMClassifier(),0.714286,0.733333,0.708333
5,AdaBoostClassifier(),0.571429,0.577778,0.562500
6,MLPClassifier(),0.571429,0.577778,0.562500
7,SGDClassifier(),0.575000,0.566667,0.497436
8,KNeighborsClassifier(),0.520833,0.522222,0.497436
9,GaussianNB(),0.484848,0.488889,0.475000


## MLP

### Build model

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.losses import MeanSquaredError

2025-07-15 15:51:48.351278: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
def Model(input_dim, output_dim):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(512, kernel_regularizer=regularizers.l1_l2(1e-6)),
        BatchNormalization(),
        LeakyReLU(0.01),
        Dropout(0.4),

        Dense(256, kernel_regularizer=regularizers.l1_l2(1e-6)),
        BatchNormalization(),
        LeakyReLU(0.01),
        Dropout(0.3),

        Dense(128, kernel_regularizer=regularizers.l1_l2(1e-6)),
        BatchNormalization(),
        LeakyReLU(0.01),
        Dropout(0.2),

        Dense(output_dim, activation='sigmoid')  # sigmoid for multi-label
    ])

    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=1e-6),
        metrics=['accuracy']
    )
    return model

In [18]:
model = Model(input_dim=X.shape[1], output_dim=1)

### Train

In [19]:
model.fit(X_mint_train, y_mint_train, validation_split=0.2, epochs=100, batch_size=32,
                    callbacks=[
                                EarlyStopping(monitor='val_loss',
                                             patience=5,
                                             restore_best_weights=True),
                                ReduceLROnPlateau(
                                  monitor='val_loss',
                                  factor=0.5,
                                  patience=5,
                                  verbose=1)
                               ]
                    )

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 429ms/step - accuracy: 0.5263 - loss: 0.8138 - val_accuracy: 0.6364 - val_loss: 0.7044 - learning_rate: 1.0000e-06
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step - accuracy: 0.6419 - loss: 0.8046 - val_accuracy: 0.6364 - val_loss: 0.7032 - learning_rate: 1.0000e-06
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.5526 - loss: 0.7530 - val_accuracy: 0.6364 - val_loss: 0.7025 - learning_rate: 1.0000e-06
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.4792 - loss: 0.7175 - val_accuracy: 0.6364 - val_loss: 0.7017 - learning_rate: 1.0000e-06
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.6052 - loss: 0.7083 - val_accuracy: 0.6364 - val_loss: 0.7003 - learning_rate: 1.0000e-06
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.6210 - loss: 0.7607 - val_accuracy: 0.6364 - val_loss: 0.6999 - learning_rate: 1.0000e-06
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.5159 

### Predict

In [20]:
y_mint_test_prob = model.predict(X_mint_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step


In [21]:
def tune_thresholds(y_true, y_pred_prob, metric='f1'):
    y_true = np.asarray(y_true)          # Fix: convert to NumPy
    y_pred_prob = np.asarray(y_pred_prob)

    best_thresholds = []
    best_scores = []

    for i in range(y_true.shape[1]):
        label_true = y_true[:, i]
        label_probs = y_pred_prob[:, i]  # Fix here too

        thresholds = np.linspace(0.0, 1.0, 101)
        scores = []

        for t in thresholds:
            label_pred = (label_probs >= t).astype(int)
            if metric == 'f1':
                score = f1_score(label_true, label_pred, zero_division=0)
            scores.append(score)

        best_t = thresholds[np.argmax(scores)]
        best_score = np.max(scores)

        best_thresholds.append(best_t)
        best_scores.append(best_score)

        print(f"Label {i}: Best threshold = {best_t:.2f}, Best {metric} = {best_score:.4f}")

    return best_thresholds, best_scores


In [22]:
# Reshape to (n_samples, 1) for binary classification
best_thresholds, _ = tune_thresholds(y_mint_test.reshape(-1, 1), y_mint_test_prob.reshape(-1, 1))

Label 0: Best threshold = 0.00, Best f1 = 0.4444


In [23]:
y_mint_pred = (model.predict(X_mint_test) >= best_thresholds).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step


### Report

In [24]:
print(classification_report(y_mint_test, y_mint_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.29      1.00      0.44         4

    accuracy                           0.29        14
   macro avg       0.14      0.50      0.22        14
weighted avg       0.08      0.29      0.13        14



## Autoencoder + MPL

In [25]:
def Autoencoder(input_dim=256):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(128),
        LeakyReLU(0.01),

        Dense(32),
        BatchNormalization(),
        LeakyReLU(0.01),

        Dense(128),
        BatchNormalization(),
        LeakyReLU(0.01),

        Dense(input_dim, activation='sigmoid')  # sigmoid for multi-label
    ])

    model.compile(optimizer=Adam(1e-6), loss=MeanSquaredError())
    return model


In [26]:
autoencoder = Autoencoder(input_dim=X.shape[1])
autoencoder.fit(X_mint_train.toarray(), X_mint_train.toarray(), epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 199ms/step - loss: 0.2493 - val_loss: 0.2287
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.2495 - val_loss: 0.2287
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.2494 - val_loss: 0.2286
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.2493 - val_loss: 0.2286
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.2493 - val_loss: 0.2286
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.2489 - val_loss: 0.2285
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.2493 - val_loss: 0.2285
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.2487 - val_loss: 0.2285
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.2498 - val_loss: 0.2285
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.2494 - val_loss: 0.2285
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.2495 - val_loss: 0.2284
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.2490 - val_loss: 0.2284


In [27]:
X_train_encoded = autoencoder.predict(X_mint_train)
X_test_encoded = autoencoder.predict(X_mint_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step


In [28]:
model = Model(input_dim=X_train_encoded.shape[1], output_dim=1)

In [29]:
model.fit(X_train_encoded, y_mint_train, validation_split=0.2, epochs=100, batch_size=32,
                    callbacks=[
                                EarlyStopping(monitor='val_loss',
                                             patience=5,
                                             restore_best_weights=True),
                                ReduceLROnPlateau(
                                  monitor='val_loss',
                                  factor=0.5,
                                  patience=5,
                                  verbose=1)
                               ]
                    )

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 285ms/step - accuracy: 0.5630 - loss: 0.6789 - val_accuracy: 0.3636 - val_loss: 0.7110 - learning_rate: 1.0000e-06
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.5685 - loss: 0.7690 - val_accuracy: 0.3636 - val_loss: 0.7099 - learning_rate: 1.0000e-06
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5843 - loss: 0.6339 - val_accuracy: 0.3636 - val_loss: 0.7085 - learning_rate: 1.0000e-06
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.6999 - loss: 0.6288 - val_accuracy: 0.3636 - val_loss: 0.7074 - learning_rate: 1.0000e-06
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6791 - loss: 0.6373 - val_accuracy: 0.3636 - val_loss: 0.7063 - learning_rate: 1.0000e-06
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.4315 - loss: 0.7512 - val_accuracy: 0.6364 - val_loss: 0.7041 - learning_rate: 1.0000e-06
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.5000 - loss

In [30]:
y_pred_prob = model.predict(X_test_encoded)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step


In [31]:
best_thresholds, _ = tune_thresholds(y_mint_test.reshape(-1, 1), y_pred_prob.reshape(-1, 1))

Label 0: Best threshold = 0.00, Best f1 = 0.4444


In [32]:
y_pred = (model.predict(X_test) >= best_thresholds).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step


### Report

In [33]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.36      1.00      0.53         5

    accuracy                           0.36        14
   macro avg       0.18      0.50      0.26        14
weighted avg       0.13      0.36      0.19        14

